In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from yellowbrick.style.colors import resolve_colors
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist ## distance matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

plt.style.use('seaborn-whitegrid')
np.set_printoptions(precision=5, suppress=True)

In this analysis we will analyze a single-cell RNA-seq dataset compiled by the Allen Institute.
The data set contains cells from a mouse neocortex (region in the brain which governs perception and cognition).

Each row corresponds to a cell.
Each column corresponds to the normalized transcript compatibility count (TCC).
We might think of entry i, j as the level of expression of gene j in cell i.

**Goal:** Unveil hierarchical structure of the genes and discover important genes.

In [ ]:
# Import data set
X = np.load('./data/analysis2/p1/X.npy')
y = np.load('./data/analysis2/p1/y.npy')


In [ ]:
# Because of the extremely high magnitude of gene expression in only a few cell, it is common to apply a log-transform to the data.
X_transformed = np.log2(X+1)
np.round(np.max(X_transformed[:,0]), 5)


In [ ]:
pca_X = PCA().fit(X)
pca_X_transformed = PCA().fit(X_transformed)

In [ ]:
# We now get the percentage of variance explained by the first principal component for the fitted X and fitted X transformed
print('%variance explained by PC1 for X:', round(pca_X.explained_variance_ratio_[0], 5))
print('%variance explained by PC1 for transformed X:', round(pca_X_transformed.explained_variance_ratio_[0], 5))

In [ ]:
def plot_cumulative_variance_explained(fitted_data):
    n_features = fitted_data.components_.shape[0]
    plt.plot(np.arange(1, n_features+1), np.cumsum(fitted_data.explained_variance_ratio_))

    plt.title('PCA cumulative variance explained', size=15)
    plt.xticks(np.arange(0, n_features, step=round(n_features/10, -2)))
    plt.yticks(np.arange(0, 1.1, step=0.1))
    plt.xlabel('Number of components')
    plt.ylabel('% Variance explained')

    plt.show()

In [ ]:
# We plot the cumulative variance explained to get an idea of how the explained variance grows as more PCs are included
plot_cumulative_variance_explained(pca_X)
plot_cumulative_variance_explained(pca_X_transformed)

In [ ]:
# How many PC should we include to explain a given threshold of variance?
threshold = 0.85

nb_PC_X = np.where(np.cumsum(pca_X.explained_variance_ratio_) >= threshold)[0][0]+1
nb_PC_X_transformed = np.where(np.cumsum(pca_X_transformed.explained_variance_ratio_) >= threshold)[0][0]+1

print('Required #PCs to explained {} of the variance in X: {}'.format(threshold, nb_PC_X))
print('Required #PCs to explained {} of the variance in transformed X: {}'.format(threshold, nb_PC_X_transformed))


In [ ]:
# We'll now visualize the transformed data. First the raw log-transformed data, then the log-transformed data transformed by PCA
plt.scatter(X_transformed[:,0], X_transformed[:,1])
plt.title('single-cell RNA-seq transformed')
plt.show()

#Plot the data projected onto PC1 and
projected_onto_PC1 = np.matmul(X_transformed, pca_X_transformed.components_[0])
projected_onto_PC2 = np.matmul(X_transformed, pca_X_transformed.components_[1])

plt.scatter(projected_onto_PC1, projected_onto_PC2)
plt.title('single-cell RNA-seq transformed, PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
# We will now visualize the data set in two dimension by reducing it using MDS (Multidimensional Scaling)
mds_X_transformed = MDS(verbose=1, eps=1e-5).fit_transform(X_transformed)

plt.scatter(mds_X_transformed[:,0], mds_X_transformed[:,1])
plt.title('single-cell RNA-seq transformed, MDS')
plt.show()


In [ ]:
# We project the data onto the top 50 principal components
projected_onto_top50 = np.matmul(X_transformed, pca_X_transformed.components_[:50].T)

In [ ]:
# Now visualize the dataset after reducing it using t-SNE (top 50 PC)
tsne_top_PC = TSNE(n_components=2, perplexity=40).fit_transform(projected_onto_top50)

plt.scatter(tsne_top_PC[:,0], tsne_top_PC[:,1])
plt.title('single-cell RNA-seq transformed, t-SNE (perplexity=40)')
plt.show()

In [ ]:
# We define the number of clusters by looking at the data set reduced using t-SNE
n_clusters = 5
clustering = KMeans(n_clusters=n_clusters, n_init=50).fit(projected_onto_top50)
colors = np.array(resolve_colors(n_clusters, 'yellowbrick'))

In [ ]:
pca_X_top50 = PCA().fit_transform(projected_onto_top50)

plt.scatter(pca_X_top50[:,0], pca_X_top50[:,1], c=colors[clustering.labels_])
plt.title('KMeans single-cell RNA-seq transformed, PCA', size=15)
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.show()

In [ ]:
mds_X_top50 = MDS(verbose=1, eps=1e-5).fit_transform(projected_onto_top50)

plt.scatter(mds_X_top50[:,0], mds_X_top50[:,1], c=colors[clustering.labels_])
plt.title('KMeans single-cell RNA-seq transformed, MDS', size=15)

plt.show()

In [ ]:
tsne_X_top50 = TSNE(n_components=2, perplexity=40).fit_transform(projected_onto_top50)

plt.scatter(tsne_X_top50[:,0], tsne_X_top50[:,1], c=colors[clustering.labels_])
plt.title('KMeans single-cell RNA-seq transformed, PCA', size=15)

plt.show()

In [ ]:
# How many clusters should we solve for ? We'll answer that by looking at an elbow plot
plt.plot(np.arange(1, 10), [KMeans(i, n_init=50).fit(projected_onto_top50).inertia_ for i in range(1, 10)])
plt.xticks(np.arange(1, 10, step=1))
plt.title('KMeans Sum of Squares Criterion', size=15)
plt.xlabel('#Clusters')
plt.ylabel('Within Group Sum of Squares (WGSS)')

plt.show()

In [ ]:
n_clusters = 4
kmeans_top50 = KMeans(n_clusters, n_init=50).fit(projected_onto_top50)
print('WGSS for {} clusters: {:.3g}'.format(n_clusters, kmeans_top50.inertia_))

# kmeans_centroid_top50 = kmeans_top50.cluster_centers_

In [ ]:
kmeans_X_centroid = KMeans(n_clusters, n_init=50).fit(X_transformed).cluster_centers_

In [ ]:
pca_kmeans_X_centroid = PCA().fit_transform(kmeans_X_centroid)

plt.scatter(pca_kmeans_X_centroid[:,0], pca_kmeans_X_centroid[:,1])
plt.show()


In [ ]:
mds_kmeans_X_centroid = MDS(n_components=2, verbose=1, eps=1e-5).fit_transform(kmeans_X_centroid)

plt.scatter(mds_kmeans_X_centroid[:,0], mds_kmeans_X_centroid[:,1])
plt.show()

In [ ]:
pca_X = PCA().fit_transform(X)
plt.scatter(pca_X[:,0], pca_X[:,1])
plt.title('single-cell RNA-seq, PCA', size=15)
plt.show()

In [ ]:
mds_X = MDS(verbose=1, eps=1e-5).fit_transform(X)

plt.scatter(mds_X[:,0], mds_X[:,1])
plt.title('single-cell RNA-seq, MDS', size=15)
plt.show()

In [ ]:
tsne_X = TSNE(n_components=2, perplexity=40).fit_transform(X)

plt.scatter(tsne_X[:,0], tsne_X[:,1])
plt.title('KMeans single-cell RNA-seq, t-SNE', size=15)

plt.show()

# Written analysis 2
_(computations used to redact and illustrate the report)_

## Part 1 - Visualisation

In [ ]:
# As the dataset is big (805MB) we might want to work on a reduced dataset first
# X_raw = np.load('./data/analysis2/p2_unsupervised/X.npy')
X_raw = np.load('./data/analysis2/p2_unsupervised_reduced/X.npy')
print('The dataset format:', X_raw.shape)

# For the same reason as with the first dataset, we perform the log-transform on this dataset
X_transformed = np.log2(X_raw + 1)

In [ ]:
# For visualization and computations purposes we reduce the dataset using PCA
pca = PCA()
X_transformed_fitted_pca = pca.fit(X_transformed)
X_transformed_pca = pca.transform(X_transformed)

In [ ]:
plot_cumulative_variance_explained(X_transformed_fitted_pca)

In [ ]:
# We then select the number of components to keep for explaining a given threshold of the variance
threshold = 0.85
nb_components = np.where(np.cumsum(X_transformed_fitted_pca.explained_variance_ratio_) >= threshold)[0][0] + 1
print('Required number of principal components to explain {} of the variance: {}'.format(threshold, nb_components))

In [ ]:
plt.scatter(X_transformed_pca[:, 0], X_transformed_pca[:,1])
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.title('Brain cells log-transformed, PCA', size=15)
plt.savefig('./plots/analysis2/brainCells_log-transformed_PCA')
plt.show()

In [ ]:
X_transformed_mds = MDS(verbose=1, eps=1e-5)
X_transformed_mds.fit_transform(X_transformed)

plt.scatter(X_transformed_mds.embedding_[:,0], X_transformed_mds.embedding_[:,1])

plt.title('Brain cells log-transformed, MDS\nstress={}'.format(X_transformed_mds.stress_), size=15)
plt.savefig('./plots/analysis2/brainCells_log-transformed_MDS')
plt.show()

In [ ]:
perplexity = 80
X_transformed_tsne = TSNE(n_components=2, perplexity=perplexity).fit_transform(X_transformed_pca)

plt.scatter(X_transformed_tsne[:,0], X_transformed_tsne[:,1])

plt.title('Brain cells log-transformed, t-SNE (perplexity={})'.format(perplexity), size=15)
plt.savefig('./plots/analysis2/brainCells_log-transformed_tSNE')
plt.show()


In [ ]:
# elbow plot
plt.plot(range(1, 6), [KMeans(i, n_init=50).fit(X_transformed).inertia_ for i in range(1, 6)])
plt.xticks(range(1, 6))
plt.xlabel('#of clusters')
plt.ylabel('WGSS')

plt.title('KMeans Sum of Squares', size=15)
plt.savefig('./plots/analysis2/brainCells_log-transformed_ElbowPlot')
plt.show()

In [ ]:
plt.plot(range(2, 6), [silhouette_score(X_transformed, KMeans(i, n_init=50).fit(X_transformed).labels_) for i in range(2, 6)])
plt.xticks(range(2, 6))
plt.xlabel('#of clusters')
plt.ylabel('Average silhouette score')

plt.title('Average Silhouette Scores', size=15)
plt.savefig('./plots/analysis2/brainCells_log-transformed_SilhouettePlot')
plt.show()

In [ ]:
# As both the elbow and silhouette plots agree on 3 being a right number of clusters,
# we will perform KMeans with  3 clusters.
n_clusters = 3
clustering = KMeans(n_clusters=n_clusters, n_init=50)
clustering.fit(X_transformed_pca)
colors = np.array(resolve_colors(n_clusters, 'yellowbrick'))

In [ ]:
mds_brain_cells = MDS(n_components=n_clusters, eps=1e-7).fit_transform(X_transformed_pca)

plt.scatter(mds_brain_cells[:,0], mds_brain_cells[:,1], c=colors[clustering.labels_])
plt.title('KMeans, Brain cells log-transformed MDS')
plt.show()

In [ ]:
for i in range(3,10):
    clustering = KMeans(i,n_init=50)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
    visualizer = SilhouetteVisualizer(clustering, colors='yellowbrick', is_fitted=True, ax=ax1)
    visualizer.fit(X_transformed_pca)

    colors = np.array(resolve_colors(i, 'yellowbrick'))
    ax2.scatter(X_transformed_pca[:,0], X_transformed_pca[:,1],c=colors[clustering.labels_])
    ax2.axis('equal')

    #set the axis to be the same for all plots
    visualizer.finalize()
    ax1.set_xlim((-.2,.6))

    plt.savefig('./plots/analysis2/subtypesKMeans/brainCells_KMeans_{}centers'.format(i))
    plt.show()

### Hierarchical clustering
The possible sub-types information about the cells hints us about a possible hierarchy of types.

For that matter, we will perform hierarchical clustering to explore this further.

_(Method inspired by Jörn's Blog, SciPy Hierarchical Clustering and Dendrogram Tutorial - https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/)_

In [ ]:
# Linkage matrix
Z = linkage(X_transformed, 'ward')

# Cophenetic Correlation Coefficient
c, coph_dist = cophenet(Z, pdist(X_transformed))

In [ ]:
# Calculate the full dendrogram
plt.figure(figsize=(25,10))
plt.title('Hierarchical clustering of brain cells', size=18)
plt.xlabel('Brain cell index')
plt.ylabel('Distance')

dendrogram(Z, leaf_rotation=90, leaf_font_size=8)

plt.savefig('./plots/analysis2/brainCells_fullDendrogram')
plt.show()
# Due to the number of samples we have, this dendrogram is hardly readable,
# although we can already get a grasp of the approximate number of clusters.

In [ ]:
# Let's reduce the number of leaves of the dendrogram
last_p = 40  # Show only the last_p merges

plt.figure(figsize=(25, 10))
plt.title('Hierarchical clustering of brain cells (last {} merges)'.format(last_p), size=25)
plt.xlabel('Size of cluster before the {}th merge'.format(last_p), size=18)
plt.ylabel('Distance', size=18)

dendrogram(Z,
           truncate_mode='lastp',
           p=last_p,
           leaf_rotation=90,
           leaf_font_size=12,
           show_contracted=True)

threshold_distance = 1800
plt.axhline(y=threshold_distance, c='black')
plt.axhline(y=1500, c='black')

plt.savefig('./plots/analysis2/brainCells_last{}Dendrogram'.format(last_p))
plt.show()
# The little dots indicates us the distances of the previous merges.
# The lower those little dots, the lower the distances at which the merges were performed.

In [ ]:
# What if we want to automate the selection of the number of clusters ?
# One of the methods is to use an elbow method, take its derivative and chose the number of clusters for x at the max_value of the derivative (where the elbow is the strongest)
last_merges = Z[-last_p:, 2]  # For better visualization, the first value is mostly exploratory
last_rev = last_merges[::-1]
idx = np.arange(1, len(last_merges) + 1, step=1)
plt.plot(idx, last_rev, label='Elbow curve')

acceleration = np.diff(last_merges, 2)
acceleration_rev = acceleration[::-1]
plt.plot(idx[:-2]+1, acceleration_rev, label='Derivative of the elbow curve')

plt.title('Elbow plot and its derivative', size=15)
plt.xlabel('#of clusters')
plt.ylabel('Distance of the merge')

plt.legend(frameon=True)
plt.savefig('./plots/analysis2/brainCells_ElbowPlot-last{}Dendrogram'.format(last_p))
plt.show()

k = acceleration_rev.argmax() + 2
print('Automated number of clusters detected:', k)

k_s = acceleration_rev.argpartition(-4)[-4:]
print('Automated multiple numbers of clusters detected:', k_s)

#### Retrieving the clusters

In [ ]:
# If we know max_d for the merge threshold
# max_d = 1800
# clusters = fcluster(Z, max_d, criterion='distance')

# If we know k
k = 5
clusters = fcluster(Z, k, criterion='maxclust')

plt.figure(figsize=(10, 8))
plt.scatter(X_transformed_pca[:,0], X_transformed_pca[:,1], c=clusters, cmap='viridis')

plt.title('Brain cells transformed, PCA, Hierarchical clustering', size=15)
plt.savefig('./plots/analysis2/brainCells_hierarchicalClustering')
plt.show()

In [ ]:
n_clusters = 5
clustering = KMeans(n_clusters, n_init=50).fit(X_transformed)
colors = np.array(resolve_colors(n_clusters, 'yellowbrick'))

plt.figure(figsize=(10,8))
plt.scatter(X_transformed_pca[:,0], X_transformed_pca[:,1], c=colors[clustering.labels_])

plt.title('Brain cells transformed, PCA, KMeans clustering', size=15)

plt.savefig('./plots/analysis2/brainCells_KMeansClustering')
plt.show()

### Gaussian Mixture Model (GMM)


In [ ]:
# TODO

### Part 2 - Unsupervised feature selection

In [ ]:
n_clusters = 5
clustering = KMeans(n_clusters=5, n_init=50).fit(X_transformed)
labels = clustering.labels_

In [ ]:
# Separate into train/test data
X_train, X_test, y_train, y_test = train_test_split(X_transformed, labels, test_size=0.33, shuffle=True)

model = LogisticRegressionCV(cv=5, Cs=[0.001, 0.01, 0.1, 1, 10], penalty='l2', multi_class='ovr', max_iter=5000).fit(X_train, y_train)
# model = LogisticRegressionCV(cv=5, Cs=[0.001, 0.01, 0.1, 1, 10], penalty='l1', solver='liblinear', max_iter=5000, multi_class='ovr')
# model = LogisticRegressionCV(cv=5, Cs=[0.001, 0.01, 0.1, 1, 10], penalty='elasticnet', solver='saga', l1_ratios=[0.25, 0.5, 0.75], multi_class='ovr', max_iter=5000)  ## Takes a loooong time

model.fit(X_train, y_train)

In [ ]:
score1 = model.score(X_test, y_test)
print(score1)
# score2 = model.score(X_test, y_test)
# score3 = model.score(X_test, y_test)

In [ ]:
n_coef = 100
sum_coef = np.sum(np.abs(model.coef_), axis=0)
top_coef = sum_coef.argsort()[::-1][:n_coef]

random_features = np.random.choice(X_transformed.shape[1], n_coef, replace=False)
max_variance_features = np.var(X_transformed, axis=0).argsort()[::-1][:n_coef]

In [ ]:
set_features = [top_coef, random_features, max_variance_features]

scores = []
for features in set_features:
    X = X_transformed[:, np.array(features)]
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, shuffle=True)

    model = LogisticRegressionCV(cv=5, Cs=[0.001, 0.01, 0.1, 1, 10], penalty='l2', multi_class='ovr', max_iter=5000)
    model.fit(X_train, y_train)

    model_score = model.score(X_test, y_test)
    scores.append(model_score)

print(np.round(np.asarray(scores), 5))

## Influence of Hyper-Parameters

In [ ]:
# We will now explore how the visualization changes when,
# instead of using the top50 PCs on the first dataset, we use another number of PCs.
# Import data set
X = np.load('./data/analysis2/p1/X.npy')
y = np.load('./data/analysis2/p1/y.npy')

X_transformed = np.log2(X+1)

pca_X = PCA().fit(X)
pca_X_transformed = PCA().fit(X_transformed)

In [ ]:
number_PCs = [10, 50, 100, 250, 500]
for PCs in number_PCs:
    projected_X = np.matmul(X_transformed, pca_X_transformed.components_[:PCs].T)
    tsne_top_PC = TSNE(n_components=2, perplexity=40).fit_transform(projected_X)

    plt.scatter(tsne_top_PC[:,0], tsne_top_PC[:,1])
    plt.title('single-cell RNA-seq transformed projected onto {} Principal Components,\nt-SNE (perplexity=40)'.format(PCs), size=15)

    plt.savefig('./plots/analysis2/hyperParameters_tSNE/tSNE_projected-onto-{}PCs'.format(PCs))
    plt.show()

In [ ]:
n_clusters = 5
number_PCs = [10, 50, 100, 250, 500]
for PCs in number_PCs:
    projected_X = np.matmul(X_transformed, pca_X_transformed.components_[:PCs].T)

    # clustering = KMeans(n_clusters=n_clusters, n_init=50)
    # clustering.fit(projected_X)
    # colors = np.array(resolve_colors(n_clusters, 'yellowbrick'))

    mds_projected_X = MDS(verbose=1, eps=1e-5).fit_transform(projected_X)

    plt.figure(figsize=(10,8))
    plt.scatter(mds_projected_X[:,0], mds_projected_X[:,1], c=y, cmap='viridis')

    plt.title('single-cell RNA-seq transformed projected onto {} Principal Components,\nMDS'.format(PCs, n_clusters), size=15)
    plt.savefig('./plots/analysis2/MDS_influence_nb-PC/MDS_{}PC'.format(PCs))

    plt.show()

In [ ]:
# Explore the effect of the perplexity parameter on t-SNE
projected_X = np.matmul(X_transformed, pca_X_transformed.components_[:50].T)

perplexities = [0, 2, 5, 20, 30, 50, 100]
for perplexity in perplexities:
    tsne = TSNE(n_components=2, perplexity=perplexity).fit_transform(projected_X)

    plt.scatter(tsne[:,0], tsne[:,1], c=y, cmap='viridis')
    plt.title('single-cell RNA-seq transformed projected onto 50 Principal Components,\nt-SNE (perplexity={})'.format(perplexity), size=15)

    plt.savefig('./plots/analysis2/t-SNE_influence-perplexity/t-SNE_perplexity{}'.format(perplexity))
    plt.show()

In [ ]:
# Explore the effect of the learning rate parameter on t-SNE
projected_X = np.matmul(X_transformed, pca_X_transformed.components_[:50].T)

learning_rates = [10, 50, 100, 250, 500, 1000]
for lr in learning_rates:
    tsne = TSNE(n_components=2, perplexity=30, learning_rate=lr)
    tsne_X = tsne.fit_transform(projected_X)

    plt.scatter(tsne_X[:,0], tsne_X[:,1], c=y, cmap='viridis')
    plt.title('single-cell RNA-seq transformed projected onto 50 Principal Components,\nt-SNE (perplexity=50, learning rate={})'.format(lr), size=15)

    # plt.savefig('./plots/analysis2/t-SNE_influence-learningRate/t-SNE_learningRate{}'.format(lr))
    plt.show()